In [6]:
import pandas as pd

In [10]:
df = pd.read_csv('students.csv')
df = df.drop(columns=['id', 'created_at', 'updated_at'])
df

,year,filiere,nom,prenom,nom_ar,prenom_ar,cne,cin,apogee,img_path
0,2035/2036,Génie Informatique,EL BASHIRIAN,LEATHA,السيف,امين,W918564999,B670005,44437015,W918564999_B670005.png
1,2021/2022,Génie Mécanique,IBN REINGER,MAKAYLA,المطرفي,ندين,X637766242,J408277,21948007,X637766242_J408277.png
2,2032/2033,Mathématiques,IBN GRAHAM,LYDA,العتيبي,مخلص,D045608345,B801076,54390120,D045608345_B801076.png
3,2026/2027,Génie Civil,PADBERG,KELLEN,السمير,عبد الرحمان,A442733990,T957599,26583236,A442733990_T957599.png
4,2030/2031,Génie Civil,EL ORN,PORTER,الخالدي,دنيا,V650249772,F625533,31184888,V650249772_F625533.png
...,...,...,...,...,...,...,...,...,...,...
9995,2028/2029,Sociologie,HANSEN,CASEY,سقا,حميد,I431109342,G323447,71167064,I431109342_G323447.png
9996,2025/2026,Psychologie,BEN OSINSKI,GERDA,المطرفي,لورينا,W244994476,P111721,15128454,W244994476_P111721.png
9997,2030/2031,Génie Informatique,IBN ROSENBAUM,SCARLETT,هوساوي,شاهر,W523810004,U564665,11586483,W523810004_U564665.png
9998,2033/2034,Génie Informatique,REILLY,DOLORES,المقبل,ليلى,W324529121,W366822,73067063,W324529121_W366822.png


In [12]:
from bidi.algorithm import get_display
import arabic_reshaper
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import os

# Paths
template_path = "fsbm_card_template.png"
output_folder = "output_cards"
os.makedirs(output_folder, exist_ok=True)

# Load fonts
font = ImageFont.truetype("fontsj/calibri.ttf", 22)
font_ar = font  # Or replace with Arabic-specific font if needed

fill_color = "#212120"

# Coordinates
positions = {
    "year": (110, 130),
    "filiere": (160, 195),
    "nom": (160, 230),
    "prenom": (160, 303),
    "cne": (130, 363),
    "cin": (146, 400),
    "apogee": (580, 400),
    "nom_ar": (428, 264),       # right-aligned anchor point
    "prenom_ar": (428, 330),    # right-aligned anchor point
}

# Reshape Arabic for RTL
def reshape_arabic(text):
    reshaped = arabic_reshaper.reshape(text)
    return get_display(reshaped)

# Draw Arabic right-aligned text
def draw_rtl_text(draw, position, text, font, fill):
    reshaped_text = reshape_arabic(text)
    bbox = font.getbbox(reshaped_text)  # (x0, y0, x1, y1)
    text_width = bbox[2] - bbox[0]
    rtl_position = (position[0] - text_width, position[1])
    draw.text(rtl_position, reshaped_text, font=font, fill=fill)


# Loop and generate cards
for _, row in df.iterrows():
    card = Image.open(template_path).convert("RGBA")
    draw = ImageDraw.Draw(card)

    # Draw fields
    draw.text(positions["year"], row["year"], fill=fill_color, font=font)
    draw.text(positions["filiere"], row["filiere"], fill=fill_color, font=font)
    draw.text(positions["nom"], row["nom"], fill=fill_color, font=font)
    draw.text(positions["prenom"], row["prenom"], fill=fill_color, font=font)
    draw.text(positions["cne"], row["cne"], fill=fill_color, font=font)
    draw.text(positions["cin"], row["cin"], fill=fill_color, font=font)
    draw.text(positions["apogee"], str(row["apogee"]), fill=fill_color, font=font)

    # Arabic fields (right-aligned)
    draw_rtl_text(draw, positions["nom_ar"], row["nom_ar"], font_ar, fill_color)
    draw_rtl_text(draw, positions["prenom_ar"], row["prenom_ar"], font_ar, fill_color)

    # Save the final image
    filename = f"{row['img_path']}.png"
    card.save(os.path.join(output_folder, filename))

    
    